In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# objects to build workflow using graphs
from langgraph.graph import StateGraph, MessagesState, START

# in memory checkpointer that enables the app
# to remember past conversations
from langgraph.checkpoint.memory import MemorySaver

ret = load_dotenv()

# returns True if .env loaded successfully
print(ret)

True


In [2]:
model = ChatOpenAI(model_name="gpt-4o-mini")
model_response = model.invoke("Tell me about Dexcom Stelo")
print(model_response)

content='As of my last knowledge update in October 2021, Dexcom had announced the Dexcom G7 continuous glucose monitoring (CGM) system, but there was no specific information available about a product called "Dexcom Stelo." It’s possible that "Dexcom Stelo" could refer to a new product or innovation that has been introduced after that date or could be a misunderstanding or miscommunication of another product name.\n\nDexcom is known for its advancements in CGM technology, which helps people with diabetes monitor their glucose levels in real-time. The devices typically consist of a small sensor inserted under the skin that measures glucose levels and transmits the data to a smartphone or other device.\n\nIf "Dexcom Stelo" is a recent development, I recommend checking the official Dexcom website or credible news sources for the most accurate and up-to-date information. Please let me know if there\'s something specific you would like to know!' additional_kwargs={'refusal': None} response_m

In [19]:
# define the function that calls the model
def call_model(state: MessagesState):
    # state = current state of the graph, which is a list of messages
    # state["messages"] is a list of all historical messages
    updated_messages = model.invoke(state["messages"])

    # models response appended to the existing list of messages
    return {"messages": updated_messages}

In [20]:
# workflow is a stategraph, keeps track of the state of the application
# we need to use a schema, MessagesState helps to keep track a list
# of messages. in other words, we are saying the state the worflow
# gonna keep track of is a list of messages
workflow = StateGraph(MessagesState)

# nodes take as input the current state of the graph
# they do something and then they update the state
workflow.add_node("model_node", call_model)

# after START, run the model node which calls the model
# add an edge from the START node to the model node
# edges determine which node to execute next, based on current state
workflow.add_edge(START, "model_node")

In [21]:
# real apps will use Postgres or SQlite checkpointers
# for experimentation use in memory checkpointer
# checkpointers store snapshots of past conversation
memory = MemorySaver()
app = workflow.compile(memory)

In [22]:
# app is now a runnable object so has invoke method
# because schema used is MessagesState, input expected
# is dict of {'messages': ""}

chat1 = {'configurable': {'thread_id': 1}}
app.invoke({"messages": "Hi My name is Mahmud!"}, config=chat1)

{'messages': [HumanMessage(content='Hi My name is Mahmud!', additional_kwargs={}, response_metadata={}, id='1d7649b1-a1c8-42c3-83e3-78893b37f699'),
  AIMessage(content='Hi Mahmud! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'stop', 'logprobs': None}, id='run-32306904-4535-4905-a7ff-2c2ab97ba71d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [23]:
# look at the reply. the whole conversation is here
app.invoke({"messages": "What is my name!"}, config=chat1)

{'messages': [HumanMessage(content='Hi My name is Mahmud!', additional_kwargs={}, response_metadata={}, id='1d7649b1-a1c8-42c3-83e3-78893b37f699'),
  AIMessage(content='Hi Mahmud! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'stop', 'logprobs': None}, id='run-32306904-4535-4905-a7ff-2c2ab97ba71d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='What is my name!', additional_kwargs={}, response_metadata={}, 

In [24]:
output = app.invoke(None, config=chat1)
for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi My name is Mahmud!
================================== Ai Message ==================================

Hi Mahmud! How can I assist you today?
================================ Human Message =================================

What is my name!
================================== Ai Message ==================================

Your name is Mahmud! How can I help you further?


In [25]:
# can the app support multiple independant conversations?
chat2 = {'configurable': {'thread_id': 2}}
app.invoke({"messages": "What is my name!"}, config=chat2)["messages"][-1].content

"I don't know your name. If you tell me, I can use it in our conversation!"

This time the app doesnt know my name confirming this is an entirely different chat

In [26]:
# make the app more interactive
def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")


In [27]:
chatbot(2)

Chatbot reply:

Nice to meet you, Mahmud Hasan! How can I assist you today?

Chatbot reply:

Las Vegas is famous for a variety of reasons, including:

1. **Casinos and Gambling**: Las Vegas is known as the gambling capital of the world, with numerous casinos offering a wide range of gaming options.

2. **Entertainment**: The city is renowned for its entertainment options, including world-class shows, concerts, magic acts, and performances by famous artists.

3. **Nightlife**: Las Vegas boasts a vibrant nightlife scene, with many nightclubs, bars, and lounges that attract visitors from around the globe.

4. **Resorts and Hotels**: The Strip is lined with extravagant hotels and resorts, each offering unique themes and attractions, such as the Bellagio, Caesars Palace, and The Venetian.

5. **Dining**: Las Vegas offers a diverse culinary scene, with numerous fine dining restaurants run by celebrity chefs, buffets, and international cuisine.

6. **Shopping**: The city features luxury shopp

# Adding a system prompt

In [28]:
# System messages are used to direct the behavior of the LLM.We will use chat prompt templates to send system messages

# ChatPromptTemplate =  a reusable structure for creating prompts for chat apps
from langchain_core.prompts import ChatPromptTemplate

In [29]:
# tuples of message types and messages
prompt = ChatPromptTemplate(
    [
        ("system", "Limit all of your responses to two sentences"),
        ("placeholder", "{messages}")  # telling the prompt template
        # is expected to get a variable messages when invoked. this variable will
        # come from the state that is passed to the call function
    ]
)

In [30]:
# state is a dict with key messages and value a list of HumanMessages
state = {"messages": ["What is the history of Delta Airlines"]}

In [31]:
# prompt also has a invoke model
prompt.invoke(state)

ChatPromptValue(messages=[SystemMessage(content='Limit all of your responses to two sentences', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the history of Delta Airlines', additional_kwargs={}, response_metadata={})])

In [32]:
model.invoke(prompt.invoke(state))

AIMessage(content='Delta Airlines was founded in 1925 as a crop-dusting service called Huff Daland Dusters. It transitioned to passenger airline operations in 1929 and expanded significantly through mergers and acquisitions, becoming one of the largest airlines in the world.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 26, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc016ee3-1218-4449-95ea-ca0ed750b1fa-0', usage_metadata={'input_tokens': 26, 'output_tokens': 50, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Chaining multiple actions

In [34]:
# this is a chain of actions = model.invoke(prompt.invoke(state))
# run the steps in order from left to right
# chains are like scikit learn pipelines
chain = prompt | model
chain.invoke(state)

AIMessage(content='Delta Airlines was founded in 1924 as a crop-dusting operation called Huff Daland Dusters and later transitioned to passenger flights in 1929. Over the decades, it expanded through numerous mergers and acquisitions, becoming one of the largest and most significant airlines in the world.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 26, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea74da98-d0d2-47c4-978b-9efbfa21f748-0', usage_metadata={'input_tokens': 26, 'output_tokens': 57, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reason

# Integrating prompt in workflow code

In [35]:
# define the function that calls the model
def call_model(state: MessagesState):
    # state = current state of the graph, which is a list of messages
    # state["messages"] is a list of all historical messages
    chain = prompt | model
    updated_messages = chain.invoke(state)

    # models response appended to the existing list of messages
    return {"messages": updated_messages}

In [36]:
workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)
workflow.add_edge(START, "model_node")
memory = MemorySaver()
app = workflow.compile(memory)

In [37]:
# now we have a chatbot that onyl gives short answerss
chatbot(2)

Chatbot reply:

Mahmud Hasan may refer to various individuals; could you specify which one or provide more context? Notable figures with that name could include politicians, academics, or athletes.

Chatbot reply:

Mahmud Hasan is a prominent Bangladeshi politician and a member of the Awami League party. He has been involved in various political activities and has served in different capacities within the government.

Chatbot reply:

The Awami League is one of the major political parties in Bangladesh, founded in 1949. It played a significant role in the country's struggle for independence and is known for advocating secularism, socialism, and democracy.

Chatbot reply:

The Bangladesh Nationalist Party (BNP) is a major political party in Bangladesh, founded in 1978 by former President Ziaur Rahman. The party is known for its nationalist ideology and has been a significant political rival to the Awami League, often advocating for conservative and pro-Islamist policies.

Chatbot reply:


KeyboardInterrupt: Interrupted by user

# Translation to Spanish

In [38]:
model = ChatOpenAI(model_name="gpt-4o-mini")

# change in the prompt
prompt = ChatPromptTemplate(
    [
        ("system", "Translate the input from English to Spanish"),
        ("placeholder", "{messages}")  # telling the prompt template
        # is expected to get a variable messages when invoked. this variable will
        # come from the state that is passed to the call function
    ]
)


def call_model(state: MessagesState):
    # state = current state of the graph, which is a list of messages
    # state["messages"] is a list of all historical messages
    chain = prompt | model
    updated_messages = chain.invoke(state)

    # models response appended to the existing list of messages
    return {"messages": updated_messages}


workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)
workflow.add_edge(START, "model_node")
memory = MemorySaver()
app = workflow.compile(memory)

In [39]:
chatbot(3)

Chatbot reply:

hola

Chatbot reply:

¿Cuál es tu nombre?

Chatbot reply:

¿Dónde está México?

Goodbye


In [40]:
# translate to any language
model = ChatOpenAI(model_name="gpt-4o-mini")

# change in the prompt
# add a variable called language to the prompt template
prompt = ChatPromptTemplate(
    [
        ("system", "Translate the input from English to {language}"),
        ("placeholder", "{messages}")
    ]
)


# %% ##################################################################
# MessagesState objects only have a single key called mesages
# we need to extend this class to also add another key language
# which will be passed to app.invoke eventually
class CustomState(MessagesState):
    language: str


# %% ##################################################################

def call_model(state: CustomState):
    chain = prompt | model
    updated_messages = chain.invoke(state)
    return {"messages": updated_messages}


workflow = StateGraph(CustomState)
workflow.add_node("model_node", call_model)
workflow.add_edge(START, "model_node")
memory = MemorySaver()
app = workflow.compile(memory)

In [43]:
def translatebot(language: str):
    # since there is no notion of conversation for translatebot
    # we will just set the chatid to a fixed number
    config = {"configurable": {"thread_id": 999}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input, "language": language},
                                              config=config,
                                              stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")

In [44]:
translatebot("Bengali")

Chatbot reply:

আমি ইমুকে ভালোবাসি!

Chatbot reply:

তুমি কোথায় ইমু? দয়া করে আমার কাছে এসো!!!

Goodbye


# Updating App with real time search

In [9]:

def call_model(state: MessagesState):
    updated_messages = model.invoke(state["messages"])

    # models response appended to the existing list of messages
    return {"messages": updated_messages}

In [10]:
def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")

Tools allow LLMs to perform actions and retrieve data from external sources. Tools is what we use to extend the capabilities of Langchain based apps. First tool we use is tavily-search. Search engine for LLMs. We need to create an account in Tavily and create API-key.

You tell the model that this tool is availabel for use, and you'll tell the model whatthe tool does. for a given input to the model, the model itself decides if it has to use the tool.

In [11]:
load_dotenv()

True

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=5)
tool_list = [search]

In [13]:
model = ChatOpenAI(model_name="gpt-4o-mini").bind_tools(tool_list)

In [14]:
from langgraph.prebuilt import ToolNode,tools_condition
call_tool = ToolNode(tool_list)

In [15]:
workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)
workflow.add_node("tools",call_tool)

workflow.add_edge(START, "model_node")
workflow.add_conditional_edges("model_node", tools_condition)
workflow.add_edge("tools", "model_node")

memory = MemorySaver()
app = workflow.compile(memory)

In [16]:
chatbot(4)

Chatbot reply:

The first five U.S. presidents are:

1. George Washington (1789-1797)
2. John Adams (1797-1801)
3. Thomas Jefferson (1801-1809)
4. James Madison (1809-1817)
5. James Monroe (1817-1825)

Chatbot reply:

[{"title": "Premarket Approval (PMA) - FDA", "url": "https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpma/pma.cfm?ID=P120005S091", "content": "approved in the original PMA.  Be sure to look at theoriginal PMArecord for more information.DeviceG5 Mobile Continuous Glucose Monitoring System, G6 Continuous Glucose Monitoring System, G7 Continuous Glucose MonitorinGeneric NameSensor, glucose, invasiveApplicantDEXCOM, INC.6340 SEQUENCE DRSAN DIEGO, CA 92121PMA NumberP120005Supplement NumberS091Date Received11/15/2023Decision Date01/09/2024Product CodeMDSAdvisory CommitteeClinical ChemistrySupplement TypeNormal 180 Day Track No User [...] approved in the original PMA.  Be sure to look at theoriginal PMArecord for more information.DeviceG5 Mobile Continuous Glucose Monitoring

In [ ]:
from langchain_core.messages import AIMessage,ToolMessage
import json

def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                if isinstance(chunk, AIMessage):
                    print(chunk.content, end="", flush=True)
                if isinstance(chunk, ToolMessage):
                    result_list = json.loads(chunk.content)
                    print(result_list[0])
            print("\n")